# El Paso Exploration Notebook

## Steps:

1. Add any required modules to the imports
2. Set County specific variables
3. Set path to sample dataset(s)
4. Import the dataset(s) in the order that they will be merged.


In [1]:
# imports
import datetime
import json
import pyspark
from pyspark.sql import Window
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, DoubleType, IntegerType, DateType, TimestampType, BooleanType
import quinn

In [2]:
# set county vars
county = "el-paso"
state = "TX"
fips_code = "48141"

schema_check_db = "faxdb"
schema_check_coll = "parcels_validation_test"

county_vars = {
    "county": county,
    "state": state,
    "fips_code": fips_code,
    "pipeline_run_id": "987654321",
    "pipeline_run_time": datetime.datetime.now(),
    "dataset_date": datetime.datetime.now(),
}

In [3]:
# init spark with delta, mongo support and 16g of memory
spark = (pyspark.sql.SparkSession.builder.appName(county)
        .config("spark.jars.packages", "io.delta:delta-core_2.12:0.7.0")
        .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
        .config(
            "spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog",
        )
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0")
        .config("spark.mongodb.input.uri", f"mongodb://127.0.0.1/{county}.default")
        .config("spark.mongodb.output.uri", f"mongodb://127.0.0.1/{county}.default")
        .config("spark.driver.memory", "16g")
        .getOrCreate()
)

In [4]:
# sample dataset paths (version controlled)
sample_ade_info = "./samples/2020-08-03_002061_APPRAISAL_INFO.TXT"
sample_ade_land = "./samples/2020-08-03_002061_APPRAISAL_LAND_DETAIL.TXT"
sample_ade_imp_info = "./samples/2020-08-03_002061_APPRAISAL_IMPROVEMENT_DETAIL.TXT"
# full datasets paths (not version controlled)
full_ade_info = ""
full_ade_land = ""
full_ade_imp_info = ""
# set dataset paths
path_ade_info = sample_ade_info
path_ade_land = sample_ade_land
path_ade_imp_info = sample_ade_imp_info

In [5]:
# load schema path
schema_path = f"./schema_tx_{county}.json"
with open(schema_path, "rb") as s:
    schema = StructType.fromJson(json.load(s))

In [24]:
# read dataset APPRAISAL_INFO.TXT
def _transform_ade_info():
    return (
        (spark.read.text(path_ade_info)
                .select(
                    F.trim(F.col("value").substr(1, 12)).cast("string").alias("parcel_id"),
                    F.trim(F.col("value").substr(547,50)).cast("string").alias("tax_id"),
                    F.trim(F.col("value").substr(13, 5)).alias("type"),
                    F.trim(F.col("value").substr(4460,15)).alias("street_number"),
                    F.trim(F.col("value").substr(1040,10)).alias("street_pre_direction"),
                    F.trim(F.col("value").substr(1050,50)).alias("street_name"),
                    F.trim(F.col("value").substr(1100,10)).alias("street_post_direction"),
                    F.trim(F.col("value").substr(4475,5)).alias("unit"),
                    F.trim(F.col("value").substr(1110,30)).alias("city"),
                    F.trim(F.col("value").substr(1140,10)).alias("zip"),
                    F.trim(F.col("value").substr(1150,255)).alias("legal_description"),
                    F.trim(F.col("value").substr(1686,10)).alias("legal_subdivision"),
                    F.trim(F.col("value").substr(2734,10)).alias("improvement_use_code"),
                    F.trim(F.col("value").substr(2742,10)).alias("land_use_code"),
                    (F.trim(F.col("value").substr(2772,20)).cast("double") / 10000).alias("lot_size_acres"),
                    F.trim(F.col("value").substr(4214,14)).cast("integer").alias("market_value"),
                    F.trim(F.col("value").substr(1946,15)).cast("integer").alias("assessed_value"),
                    F.to_date(F.trim(F.col("value").substr(2034,25)), "MMddyyyy").alias("deed_transfer_date"),
                    F.trim(F.col("value").substr(4492,70)).alias("oor_1"),
                    F.trim(F.col("value").substr(4562,60)).alias("oor_2"),
                    F.trim(F.col("value").substr(4622,60)).alias("oor_3"),
                    F.trim(F.col("value").substr(4682,60)).alias("oor_4"),
                    F.trim(F.col("value").substr(4742,50)).alias("oor_5"),
                    F.trim(F.col("value").substr(4792,50)).alias("oor_6"),
                    F.trim(F.col("value").substr(4847,5)).alias("oor_7"),
                    F.trim(F.col("value").substr(4852,4)).alias("oor_8"),
                    F.trim(F.col("value").substr(4842,5)).alias("oor_9")         
                )
                .where((F.col("type") == "R") | (F.col("type") == "M"))
                .withColumn(
                    "parcel_id",
                    F.regexp_replace('parcel_id', r'^[0]*', '')
                )
                .withColumn(
                    "legal_description",
                    quinn.single_space(F.col("legal_description"))
                )
                .withColumn(
                    "street_address",
                    quinn.single_space(
                        F.regexp_replace(
                            F.concat_ws(
                                " ",
                                "street_number",
                                "street_pre_direction",
                                "street_name",
                                "street_post_direction",
                                "unit",
                            ),
                            '"',
                            "",  # regexp replacement values
                        )
                    ),
                )
                .withColumn(
                    "oor",
                    quinn.single_space(
                        F.regexp_replace(
                            F.concat_ws(
                                " ",
                                "oor_1",
                                "oor_2",
                                "oor_3",
                                "oor_4",
                                "oor_5",
                                "oor_6",
                                "oor_7",
                                "oor_9",
                            ),
                            '"',
                            "",  # regexp replacement values
                        )
                    ),
                )
        ).drop("oor_1", "oor_2", "oor_3", "oor_4", "oor_5", "oor_6", "oor_7", "oor_8", "oor_9")
    )

In [25]:
df_ade_info = _transform_ade_info()

In [61]:
# load APPRAISAL_INFO.TXT to db
df_ade_info.write.format("mongo").mode("overwrite").option("collection", "df_ade_info").save()

In [27]:
# read APPRAISAL_LAND_DETAIL.TXT
def _transform_land():
    w_land = Window.partitionBy("parcel_id")
    return (
        (spark.read.text(path_ade_land)
            .select(
                F.trim(F.col("value").substr(1, 12)).cast("string").alias("parcel_id"),
                #   Incorrect values, see readme
                F.trim(F.col("value").substr(39,25)).cast("string").alias("land_desc_desc"),
                F.trim(F.col("value").substr(84,14)).cast("integer").alias("sqft"),
                F.trim(F.col("value").substr(112,14)).cast("integer").alias("lot_depth"),
                F.trim(F.col("value").substr(98,14)).cast("integer").alias("lot_frontage")
            )
            .withColumn(
                "parcel_id",
                F.regexp_replace('parcel_id', r'^[0]*', '')
            )
            .withColumn("lot_size_sqft", F.sum("sqft").over(w_land))
            .withColumn("lot_depth_ft", F.sum("lot_depth").over(w_land))
            .withColumn("lot_frontage_ft", F.sum("lot_frontage").over(w_land))
        ).drop_duplicates(["parcel_id"])
    )

In [28]:
df_ade_land = _transform_land()

In [83]:
# load APPRAISAL_LAND_DETAIL.TXT to db
df_ade_land.write.format("mongo").mode("overwrite").option("collection", "df_ade_land").save()

In [17]:
# read APPRAISAL_IMPROVEMENT_DETAIL.TXT
def _transfrom_imp_detail():
    w_impr = Window.partitionBy("parcel_id")
    return(
    
        (spark.read.text(path_ade_imp_info)
            .select(
                F.trim(F.col("value").substr(1,12)).alias("parcel_id"),
                F.trim(F.col("value").substr(86,4)).cast("integer").alias("year"),
                F.trim(F.col("value").substr(94,15)).cast("integer").alias("sqft"),
                F.trim(F.col("value").substr(41,10)).alias("imprv_det_type_cd"),
            ).filter("imprv_det_type_cd like 'MA%'")
            .withColumn(
                    "parcel_id",
                    F.regexp_replace('parcel_id', r'^[0]*', '')
            )
            .withColumn("structure_total_sqft", F.sum("sqft").over(w_impr))
            .withColumn("year_built", F.min("year").over(w_impr))
        ).drop("sqft", "year").drop_duplicates(["parcel_id"])
    )

In [18]:
df_ade_imp_info = _transfrom_imp_detail().show(truncate=False)

+---------+-----------------+--------------------+----------+
|parcel_id|imprv_det_type_cd|structure_total_sqft|year_built|
+---------+-----------------+--------------------+----------+
|100010   |MA               |7472                |2014      |
|100735   |MA               |1252                |1961      |
|100768   |MA               |1286                |2004      |
|10096    |MA               |2048                |1994      |
|100964   |MA               |678                 |1987      |
|101021   |MA               |973                 |1916      |
|101122   |MA               |2468                |2006      |
|101205   |MA               |1348                |2007      |
|101261   |MA               |672                 |1981      |
|101272   |MA               |806                 |1952      |
|102113   |MA               |2066                |1975      |
|102539   |MA               |2339                |1987      |
|102684   |MA               |1595                |2007      |
|102745 

In [16]:
df_ade_imp_info.where("parcel_id = 100010").orderBy("imprv_det_type_cd").show(truncate=False)

+---------+-----------------+--------------------+----------+
|parcel_id|imprv_det_type_cd|structure_total_sqft|year_built|
+---------+-----------------+--------------------+----------+
|100010   |MA               |7472                |2014      |
+---------+-----------------+--------------------+----------+



In [ ]:
# load APPRAISAL_IMPROVEMENT_DETAIL.TXT to db
df_ade_imp_info.write.format("mongo").mode("overwrite").option("collection", "df_ade_imp_info").save()

In [ ]:
# merge datasets (if necassary)

# Example assuming more than one dataset
df_county = df_ade_info.join(df_ade_land, "parcel_id", "left").join(df_ade_imp_info, "parcel_id", "left")

In [ ]:
# add required columns
county_parcels = df_county \
    .withColumn("county", F.lit(county_vars["county"])) \
        .withColumn(
        "fingerprint", F.base64(F.concat(F.lit(county_vars["county"]), "parcel_id"))
        ) \
        .withColumn("state", F.lit(county_vars["state"])) \
        .withColumn("fips_code", F.lit(county_vars["fips_code"])) \
        .withColumn("pipeline_run_id", F.lit(county_vars["pipeline_run_id"])) \
        .withColumn("pipeline_run_time", F.lit(county_vars["pipeline_run_time"])) \
        .withColumn("dataset_date", F.lit(county_vars["dataset_date"])) \
        .withColumn("record_hash", F.sha2(F.concat_ws("||", *df_county.columns), 256)) \
    .drop_duplicates(["fingerprint"])

In [ ]:
# validate schema

In [ ]:
# load final dataset into a mongo collection with schema validation
county_parcels.write.format("mongo").mode("append").option("database", schema_check_db).option("collection", schema_check_coll).save()

In [ ]:
# get missing fields and document why they're missing on county readme.md
missing_fields = [f for f in schema.fieldNames() if f not in county_parcels.schema.fieldNames()]
print(missing_fields)

In [ ]:
# check for extra fields
extra_fields = [f for f in county_parcels.schema.fieldNames() if f not in schema.fieldNames()]
print(extra_fields)